## Read data files

In [1]:
import pandas as pd
import numpy as np
import sys, os, csv
from collections import defaultdict
from scipy.sparse import csr_matrix, csc_matrix, diags
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

In [2]:
from keras.layers import Dense, Dropout, Input, LSTM, Embedding, Activation, Bidirectional, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
#from keras import initializers, regularizers, constraints, optimizers, layers

C:\Users\Xia_Dell\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv('Accepted_answer_prediction_data_train.txt', sep ='\t',
                 header=None,names=['C_ID','Description','U_ID','Type','Time'])
label = pd.read_csv('Accepted_answer_prediction_labels_train.txt', sep ='\t',
                 header=None,names=['C_ID','label'])
df = pd.merge(df, label, on=['C_ID'])

In [4]:
embedding_file='Sample/glove.6B.100d.txt'

In [5]:
df['length'] = [len(str(des).split()) for des in df['Description'].tolist()]

In [6]:
# shuffle data and train/test split
df = shuffle(df)
train_frac = 0.8

In [7]:
emb_D = 100 # Dimension of the embedding.  Here we chose 100
doc_len = 300 # we chose certain number of words used in the model. For documents with fewer words, zero padding will be used.

In [8]:
df.head()

,C_ID,Description,U_ID,Type,Time,label,length
2536,11946,christian christian group local church recover...,93,A,0.02,0,236
7295,18291,view follow christ christian continu fulfil ju...,491,A,0.28,0,69
4389,14417,church period preach long time fanci oratori a...,233,A,0.19,1,138
2107,11374,trend capit word bibl koran extrem simpl reaso...,72,A,0.94,0,68
9831,21673,question make sens mention triniti understand ...,713,A,0.42,0,78


## Tokenization
Prepare X and y

In [9]:
Train_Doc = df["Description"].fillna("NA").values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(Train_Doc))
Train_tokenize = tokenizer.texts_to_sequences(Train_Doc)
X_t = pad_sequences(Train_tokenize, maxlen=doc_len)
X_t.shape

(15464, 300)

In [10]:
y = df['label'].values

### Embedding
Since the given dataset has been preprocess. we will apply stemming on the embedding file

In [11]:
from gensim.parsing.porter import PorterStemmer

C:\Users\Xia_Dell\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
p = PorterStemmer()

In [13]:
embedding = {}
f = open(embedding_file, encoding="utf8")
for word_vec in f:
    word_vec = word_vec.split()
    word = word_vec[0]
    vec = np.asarray(word_vec[1:])
    embedding[p.stem(word)] = vec.astype('float32')

In [14]:
print ('the length of the embedding volcabulary after stemming treatment is ', len(embedding))

the length of the embedding volcabulary after stemming treatment is  332346


Initiate the embedding matrix with random numbers based on the mean and std of the embedding file

In [15]:
emb_values = np.stack(embedding.values())
emb_mean = emb_values.mean()
emb_std = emb_values.std()

In [26]:
vocab = tokenizer.word_index
vocab_len = len(vocab)+1 # here the reason of adding 1 is because the index of word in vocab starts at 1

In [27]:
embed_Matrix = np.random.normal(emb_mean, emb_std, (vocab_len, emb_D))
for w, i in vocab.items():
    embed_Vector = embedding.get(w)
    if embed_Vector is not None: 
        embed_Matrix[i] = embed_Vector

### LSTM model

In [32]:
from sklearn.utils import class_weight

In [35]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [41]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [70]:
a = Input(shape=(doc_len,))
b = Embedding(vocab_len, emb_D, weights=[embed_Matrix])(a)
b = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(b)
b = GlobalMaxPool1D()(b)
b = Dense(100, activation="elu")(b)
b = Dropout(0.2)(b)
b = Dense(50, activation="elu")(b)
b = Dropout(0.2)(b)
# b = Dense(20, activation="elu")(b)
# b = Dropout(0.15)(b)
b = Dense(1, activation="sigmoid")(b)
model = Model(inputs=a, outputs=b)
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=[sensitivity, specificity, 'accuracy'])

### Model training

In [74]:
model.fit(X_t, y, batch_size=32, epochs=50, validation_split=0.2,class_weight={0:class_weights[0],1:class_weights[1]});

Train on 12371 samples, validate on 3093 samples
Epoch 1/50
12371/12371 [==============================] - 339s 27ms/step - loss: 0.6239 - sensitivity: 0.6581 - specificity: 0.6502 - acc: 0.6501 - val_loss: 0.7892 - val_sensitivity: 0.1811 - val_specificity: 0.8983 - val_acc: 0.7459
Epoch 2/50
12371/12371 [==============================] - 324s 26ms/step - loss: 0.5587 - sensitivity: 0.7008 - specificity: 0.7113 - acc: 0.7085 - val_loss: 0.7681 - val_sensitivity: 0.6098 - val_specificity: 0.5513 - val_acc: 0.5613
Epoch 3/50
12371/12371 [==============================] - 285s 23ms/step - loss: 0.4696 - sensitivity: 0.7793 - specificity: 0.7661 - acc: 0.7687 - val_loss: 0.9921 - val_sensitivity: 0.3010 - val_specificity: 0.7765 - val_acc: 0.6760
Epoch 4/50
12371/12371 [==============================] - 337s 27ms/step - loss: 0.3598 - sensitivity: 0.8643 - specificity: 0.8283 - acc: 0.8340 - val_loss: 1.3246 - val_sensitivity: 0.2143 - val_specificity: 0.8252 - val_acc: 0.6977
Epoch 5/50


12371/12371 [==============================] - 191s 15ms/step - loss: 0.0238 - sensitivity: 0.9837 - specificity: 0.9901 - acc: 0.9905 - val_loss: 4.5227 - val_sensitivity: 0.2068 - val_specificity: 0.8222 - val_acc: 0.6925
Epoch 36/50
12371/12371 [==============================] - 188s 15ms/step - loss: 0.0196 - sensitivity: 0.9931 - specificity: 0.9927 - acc: 0.9930 - val_loss: 5.2660 - val_sensitivity: 0.1526 - val_specificity: 0.8812 - val_acc: 0.7262
Epoch 37/50
12371/12371 [==============================] - 185s 15ms/step - loss: 0.0110 - sensitivity: 0.9929 - specificity: 0.9962 - acc: 0.9964 - val_loss: 4.4053 - val_sensitivity: 0.2442 - val_specificity: 0.7619 - val_acc: 0.6537
Epoch 38/50
12371/12371 [==============================] - 182s 15ms/step - loss: 0.0180 - sensitivity: 0.9927 - specificity: 0.9945 - acc: 0.9945 - val_loss: 4.2194 - val_sensitivity: 0.2631 - val_specificity: 0.7490 - val_acc: 0.6469
Epoch 39/50
12371/12371 [==============================] - 181s 15ms

In [72]:
from sklearn.model_selection import StratifiedKFold

7.5

In [73]:
8*50/60

6.666666666666667